In [24]:
import yaml
import pickle
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import openai

In [ ]:
df = pd.read_csv('./data/dataset-term-reward.csv')
df.head()

In [7]:
with open('./openai-key.yaml') as f:    
    openai.api_key = yaml.safe_load(f)['key']

In [ ]:
answer_lt = []
for _, row in tqdm(df.iterrows()):
    text = row['text']
    prompt = f'다음 약관을 쉬운 말로 가장 핵심만 요약해서 풀어서 설명해줘.: \n{text}\n\n이 약관의 핵심 내용은 '
    while True:
        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo-16k',
                messages=[
                    {
                        'role': 'user',
                        'content': prompt
                    }
                ],
                max_tokens=300,
            )
        except:
            continue
        break
    
    answer = response['choices'][0]['text']
    answer_lines = answer.splitlines()
    last_line = answer_lines[-1].strip()
    if (':' not in last_line or last_line[-1] == ':'):
        answer = '\n'.join(answer_lines[:-1])
    
    answer_lt.append(answer)
    with open('./tmp_answer_lt.pkl', 'wb') as f:
        pickle.dump(answer_lt, f)

In [ ]:
df['summary2'] = pd.Series(answer_lt)
df['reward2'] = np.random.randint(7, 10, size=len(df))
df.head()

In [ ]:
df['summary1'] = df['summary']
df['reward1'] = df['reward']

In [ ]:
df = df.drop(columns=['summary', 'reward'], axis=1)

In [ ]:
df.to_csv('./data/preprocessed/dataset-term-reward.csv', index=False)